In [1]:
import numpy as np
import pandas as pd

from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm
import ast

import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns  

# seaborn settings
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)

import altair as alt
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
filePath = r'https://raw.githubusercontent.com/hdm-statistik-lpj/project/main/data/external/immobilienNuernberg_cleaned.csv'
df = pd.read_csv(filePath)

In [3]:
# Nummerische Variablen
df['constructionYear'] = df['constructionYear'].astype(int)
df['livingSpace'] = df['livingSpace'].astype(float)
df['photosCount'] = df['photosCount'].astype(int)
df['localAmenitiesScore'] = df['localAmenitiesScore'].astype(float)
df['localMobilityScore'] = df['localMobilityScore'].astype(float)
df['latitudeDistrict'] = df['latitudeDistrict'].astype(float)
df['longitudeDistrict'] = df['longitudeDistrict'].astype(float)
df['distanceToCenter'] = df['distanceToCenter'].astype(float)
df['price'] = df['price'].astype(float)
df['rooms'] = df['rooms'].astype(float)

# Kategorische Variablen
df['district'] = df['district'].astype('category')
df['location'] = df['location'].astype('category')
df['energyClass'] = df['energyClass'].astype('category')
df['energySources'] = df['energySources'].astype('category')

# Binäre Variablen
binaryColumns = ['elevator', 'sauna', 'swimmingpool', 'kitchenEqipped', 
                  'cellar', 'storageRoom', 'wheelchairAccessible', 'assistedLiving', 
                  'parkingSlots', 'guestToilet', 'privateOutdoorSpace', 
                  'chimney', 'attics', 'energyEfficiency', 
                  'Northeast', 'Northwest', 'Southeast', 'Southwest']

for col in binaryColumns:
    df[col] = df[col].astype('category')

In [4]:
# Modell 1: 'price' als abhängige Variable
X_price = df.drop('price', axis=1)
y_price = df['price']
X_price['log_livingSpace'] = np.log(X_price['livingSpace'])

In [5]:
# Aufteilen der Daten für das Modell mit 'price' als abhängige Variable
X_train_price, X_test_price, y_train_price, y_test_price = train_test_split(
    X_price, y_price, test_size=0.2, random_state=42, stratify=df['energyClass'])
df_train_price = pd.concat([X_train_price, y_train_price], axis=1)
df_test_price = pd.concat([X_test_price, y_test_price], axis=1)

In [6]:
df_train_price['log_price'] = np.log(df_train_price['price'])
df_train_price['log_livingSpace'] = np.log(df_train_price['livingSpace'])

In [7]:
# Modell 1: 'price' als abhängige Variable
X_price = df.drop('price', axis=1)
y_price = df['price']
X_price['log_livingSpace'] = np.log(X_price['livingSpace'])

In [8]:
# Log-Modelle
modelFormulas_log = [
    'np.log(price) ~ log_livingSpace + distanceToCenter + location + constructionYear + energyClass + energySources + elevator + sauna + swimmingpool + kitchenEqipped + cellar + storageRoom + wheelchairAccessible + assistedLiving + parkingSlots + privateOutdoorSpace + chimney + guestToilet + attics',
    'np.log(price) ~ log_livingSpace + distanceToCenter + location + constructionYear + energyClass + elevator + sauna + swimmingpool + kitchenEqipped + cellar + storageRoom + wheelchairAccessible + assistedLiving + parkingSlots + privateOutdoorSpace + chimney + guestToilet + attics',
    'np.log(price) ~ log_livingSpace + distanceToCenter + location + constructionYear + energyClass + sauna + swimmingpool + kitchenEqipped + cellar + storageRoom + wheelchairAccessible + assistedLiving + parkingSlots + privateOutdoorSpace + chimney + guestToilet + attics',
    'np.log(price) ~ log_livingSpace + distanceToCenter + location + constructionYear + energyClass + sauna + kitchenEqipped + cellar + storageRoom + wheelchairAccessible + assistedLiving + parkingSlots + privateOutdoorSpace + chimney + guestToilet + attics',
    'np.log(price) ~ log_livingSpace + distanceToCenter + location + constructionYear + energyClass + sauna + kitchenEqipped + cellar + storageRoom + wheelchairAccessible + parkingSlots + privateOutdoorSpace + chimney + guestToilet + attics',
    'np.log(price) ~ log_livingSpace + distanceToCenter + location + constructionYear + energyClass + sauna + kitchenEqipped + cellar + storageRoom + wheelchairAccessible + parkingSlots + privateOutdoorSpace + chimney + attics',
    'np.log(price) ~ log_livingSpace + distanceToCenter + location + constructionYear + energyClass + sauna + kitchenEqipped + cellar + storageRoom + wheelchairAccessible + parkingSlots + chimney + attics',
    'np.log(price) ~ log_livingSpace + distanceToCenter + location + constructionYear + energyClass + sauna + kitchenEqipped + cellar + storageRoom + wheelchairAccessible + parkingSlots + attics',
    'np.log(price) ~ log_livingSpace + distanceToCenter + location + constructionYear + energyClass + sauna + kitchenEqipped + cellar + storageRoom + parkingSlots + attics',
    'np.log(price) ~ log_livingSpace + distanceToCenter + location + constructionYear + energyClass + sauna + kitchenEqipped + cellar + storageRoom + attics',
    'np.log(price) ~ log_livingSpace + location + constructionYear + energyClass + sauna + kitchenEqipped + cellar + storageRoom + attics',
    'np.log(price) ~ log_livingSpace + location + constructionYear + energyClass + sauna + kitchenEqipped + cellar + attics',
]

In [9]:
# DataFrame für Koeffizienten und p-Werte
modelSummary = pd.DataFrame()

# DataFrame für R-Quadrat und angepasstes R-Quadrat
modelFitSummary = pd.DataFrame(columns=['Model', 'R-Squared', 'Adj. R-Squared'])

# Index setzen
firstModel = smf.ols(modelFormulas_log[0], data=df_train_price).fit()
modelSummary['Coefficient_Model_1'] = firstModel.params
modelSummary['P-Value_Model_1'] = firstModel.pvalues
firstFitSummary = pd.DataFrame({'Model': ['Model 1'],
                                  'R-Squared': [firstModel.rsquared],
                                  'Adj. R-Squared': [firstModel.rsquared_adj]})
modelFitSummary = pd.concat([modelFitSummary, firstFitSummary])

# Restliche  Modelle durchlaufen
for i, formula in enumerate(modelFormulas_log[1:], start=2):
    model = smf.ols(formula, data=df_train_price).fit()
    modelSummary[f'Coefficient_Model_{i}'] = model.params
    modelSummary[f'P-Value_Model_{i}'] = model.pvalues
    fitSummary = pd.DataFrame({'Model': [f'Model {i}'],
                                'R-Squared': [model.rsquared],
                                'Adj. R-Squared': [model.rsquared_adj]})
    modelFitSummary = pd.concat([modelFitSummary, fitSummary])

# Ergebnisse anzeigen
print(modelSummary.T)
print(modelFitSummary.T)


modelSummary_markdown = modelSummary.to_markdown()
modelFitSummary_markdown = modelFitSummary.to_markdown()

C:\Users\papad\AppData\Local\Temp\ipykernel_12804\3193224927.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  modelFitSummary = pd.concat([modelFitSummary, firstFitSummary])


                         Intercept  location[T.2]  location[T.3]  \
Coefficient_Model_1   4.655844e+00      -0.061721  -1.508127e-01   
P-Value_Model_1       1.829251e-08       0.056626   1.530184e-07   
Coefficient_Model_2   4.775391e+00      -0.062333  -1.524139e-01   
P-Value_Model_2       1.646377e-09       0.044984   2.694161e-08   
Coefficient_Model_3   4.780119e+00      -0.062369  -1.524449e-01   
P-Value_Model_3       2.297118e-10       0.044251   2.453346e-08   
Coefficient_Model_4   4.778100e+00      -0.062281  -1.524237e-01   
P-Value_Model_4       2.211005e-10       0.044260   2.379418e-08   
Coefficient_Model_5   4.765487e+00      -0.062560  -1.530982e-01   
P-Value_Model_5       2.159175e-10       0.042987   1.729238e-08   
Coefficient_Model_6   4.731445e+00      -0.061883  -1.536054e-01   
P-Value_Model_6       2.161220e-10       0.044794   1.462575e-08   
Coefficient_Model_7   4.823312e+00      -0.062381  -1.532447e-01   
P-Value_Model_7       5.241506e-11       0.04295

In [11]:
print(modelSummary_markdown)
print(modelFitSummary_markdown)

|                           |   Coefficient_Model_1 |   P-Value_Model_1 |   Coefficient_Model_2 |   P-Value_Model_2 |   Coefficient_Model_3 |   P-Value_Model_3 |   Coefficient_Model_4 |   P-Value_Model_4 |   Coefficient_Model_5 |   P-Value_Model_5 |   Coefficient_Model_6 |   P-Value_Model_6 |   Coefficient_Model_7 |   P-Value_Model_7 |   Coefficient_Model_8 |   P-Value_Model_8 |   Coefficient_Model_9 |   P-Value_Model_9 |   Coefficient_Model_10 |   P-Value_Model_10 |   Coefficient_Model_11 |   P-Value_Model_11 |   Coefficient_Model_12 |   P-Value_Model_12 |
|:--------------------------|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|-----------

|                           |   Coefficient_Model_1 |   P-Value_Model_1 |   Coefficient_Model_2 |   P-Value_Model_2 |   Coefficient_Model_3 |   P-Value_Model_3 |   Coefficient_Model_4 |   P-Value_Model_4 |   Coefficient_Model_5 |   P-Value_Model_5 |   Coefficient_Model_6 |   P-Value_Model_6 |   Coefficient_Model_7 |   P-Value_Model_7 |   Coefficient_Model_8 |   P-Value_Model_8 |   Coefficient_Model_9 |   P-Value_Model_9 |   Coefficient_Model_10 |   P-Value_Model_10 |   Coefficient_Model_11 |   P-Value_Model_11 |   Coefficient_Model_12 |   P-Value_Model_12 |
|:--------------------------|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|-----------------------:|-------------------:|-----------------------:|-------------------:|-----------------------:|-------------------:|
| Intercept                 |            4.65584    |      1.82925e-08  |           4.77539     |      1.64638e-09  |            4.78012    |      2.29712e-10  |            4.7781     |      2.211e-10    |            4.76549    |      2.15917e-10  |            4.73144    |      2.16122e-10  |            4.82331    |      5.24151e-11  |            4.77209    |      7.14953e-11  |             4.80355   |      5.4631e-11   |             4.57146    |       1.57866e-10  |             4.75196    |       2.09525e-11  |             4.51341    |       1.06688e-10  |
| location[T.2]             |           -0.0617205  |      0.0566265    |          -0.062333    |      0.0449844    |           -0.0623694  |      0.0442505    |           -0.062281   |      0.04426      |           -0.0625596  |      0.0429874    |           -0.0618832  |      0.0447936    |           -0.0623808  |      0.0429551    |           -0.0617754  |      0.0449212    |            -0.0615526 |      0.0458529    |            -0.0603698  |       0.0503723    |            -0.0587775  |       0.0571248    |            -0.0545418  |       0.0775752    |
| location[T.3]             |           -0.150813   |      1.53018e-07  |          -0.152414    |      2.69416e-08  |           -0.152445   |      2.45335e-08  |           -0.152424   |      2.37942e-08  |           -0.153098   |      1.72924e-08  |           -0.153605   |      1.46258e-08  |           -0.153245   |      1.54153e-08  |           -0.151659   |      1.99043e-08  |            -0.153059  |      1.48312e-08  |            -0.155358   |       8.99845e-09  |            -0.165738   |       3.72515e-10  |            -0.167018   |       3.05644e-10  |
| location[T.4]             |           -0.117202   |      5.75786e-06  |          -0.116462    |      3.00107e-06  |           -0.11646    |      2.92862e-06  |           -0.116407   |      2.87636e-06  |           -0.116993   |      2.28055e-06  |           -0.11713    |      2.17046e-06  |           -0.114897   |      2.77709e-06  |           -0.113086   |      3.65022e-06  |            -0.110594  |      5.59062e-06  |            -0.112727   |       3.66274e-06  |            -0.119033   |       8.25853e-07  |            -0.114745   |       1.91508e-06  |
| energyClass[T.2]          |           -0.0249112  |      0.775023     |          -0.0222525   |      0.733061     |           -0.0221844  |      0.733209     |           -0.0228344  |      0.724443     |           -0.0236231  |      0.714759     |           -0.0249246  |      0.699202     |           -0.0259389  |      0.687382     |           -0.0259667  |      0.687015     |            -0.039712  |      0.532585     |            -0.0403551  |       0.526475     |            -0.0240955  |       0.702664     |            -0.0392013  |       0.532724     |
| energyClass[T.3]          |           -0.15181    |      0.0532495    |          -0.133482    |      0.00735519   |           -0.133442   |      0.00726053   |           -0.133422   |      0.00720698   |           -0.13312    |      0.00726337   |           -0.134076   |      0.00674935   |           -0.132453   |      0.00735262   |           -0.137262   |      0.00521893   |            -0.13818   |      0.00495474   |            -0.142952   |       0.00362693   |            -0.122823   |       0.0101139    |            -0.123777   |       0.00975744   |
| energyClass[T.4]          |           -0.398778   |      4.02752e-07  |          -0.387716    |      3.83255e-14  |           -0.387643   |      3.12817e-14  |           -0.387603   |      2.95139e-14  |           -0.386978   |      2.8467e-14   |           -0.387789   |      2.30242e-14  |           -0.388902   |      1.85117e-14  |           -0.38716    |      2.25195e-14  |            -0.386011  |      2.73392e-14  |            -0.383319   |       4.12656e-14  |            -0.377911   |       8.85497e-14  |            -0.388929   |       1.4863e-14   |
| energyClass[T.5]          |           -0.420036   |      7.16068e-08  |          -0.414725    |      8.11231e-17  |           -0.414666   |      6.67831e-17  |           -0.414753   |      6.05764e-17  |           -0.414758   |      5.63449e-17  |           -0.416452   |      3.163e-17    |           -0.416249   |      3.13991e-17  |           -0.415392   |      3.4983e-17   |            -0.41519   |      3.77582e-17  |            -0.413561   |       5.20059e-17  |            -0.404098   |       1.71505e-16  |            -0.411817   |       4.78538e-17  |
| energyClass[T.6]          |           -0.426341   |      9.19684e-08  |          -0.413834    |      6.66431e-15  |           -0.413786   |      5.87812e-15  |           -0.41388    |      5.39098e-15  |           -0.414068   |      4.90172e-15  |           -0.415728   |      3.06465e-15  |           -0.416828   |      2.46843e-15  |           -0.414644   |      3.11455e-15  |            -0.416306  |      2.50928e-15  |            -0.417872   |       2.12276e-15  |            -0.409991   |       5.75773e-15  |            -0.419749   |       1.26389e-15  |
| energyClass[T.7]          |           -0.415415   |      1.13338e-06  |          -0.399175    |      1.27177e-11  |           -0.399007   |      7.48936e-12  |           -0.399012   |      7.11262e-12  |           -0.399121   |      6.68404e-12  |           -0.400815   |      4.67747e-12  |           -0.403583   |      2.97629e-12  |           -0.407286   |      1.68542e-12  |            -0.406641  |      1.87664e-12  |            -0.406447   |       2.03172e-12  |            -0.390546   |       7.55279e-12  |            -0.396621   |       3.94032e-12  |
| energyClass[T.8]          |           -0.366473   |      0.00012324   |          -0.346903    |      1.72311e-06  |           -0.346745   |      1.46304e-06  |           -0.346838   |      1.4147e-06   |           -0.347096   |      1.35348e-06  |           -0.348795   |      1.13318e-06  |           -0.350584   |      9.7494e-07   |           -0.34866    |      1.09667e-06  |            -0.3489    |      1.09738e-06  |            -0.350507   |       1.00847e-06  |            -0.342437   |       1.74408e-06  |            -0.356432   |       6.27962e-07  |
| energyClass[T.9]          |           -0.385439   |      0.0186873    |          -0.36611     |      0.0143908    |           -0.366018   |      0.0142619    |           -0.366101   |      0.014136     |           -0.366159   |      0.014024     |           -0.368216   |      0.0133642    |           -0.371395   |      0.0125097    |           -0.368746   |      0.013115     |            -0.370838  |      0.0126694    |            -0.36933    |       0.0131429    |            -0.355065   |       0.0171459    |            -0.367014   |       0.0139653    |
| energySources[T.2.0]      |           -0.017066   |      0.518188     |         nan           |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| energySources[T.3.0]      |           -0.0430195  |      0.186811     |         nan           |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| energySources[T.4.0]      |           -0.0322047  |      0.552454     |         nan           |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| energySources[T.5.0]      |            0.0242932  |      0.817729     |         nan           |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| energySources[T.6.0]      |            0.066872   |      0.484054     |         nan           |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| energySources[T.7.0]      |            0.0374962  |      0.619597     |         nan           |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| energySources[T.8.0]      |           -0.0511404  |      0.550996     |         nan           |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| energySources[T.9.0]      |           -0.0958102  |      0.343513     |         nan           |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| energySources[T.10.0]     |            0.0685031  |      0.450945     |         nan           |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| energySources[T.11.0]     |            0.0689547  |      0.746487     |         nan           |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| energySources[T.12.0]     |           -0.164114   |      0.255859     |         nan           |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| energySources[T.13.0]     |           -0.0473604  |      0.767257     |         nan           |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| energySources[T.14.0]     |            0.111474   |      0.581368     |         nan           |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| energySources[T.15.0]     |           -0.102057   |      0.609634     |         nan           |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| energySources[T.16.0]     |           -1.1471e-16 |      0.526018     |         nan           |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| energySources[T.18.0]     |           -0.0401042  |      0.851675     |         nan           |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| energySources[T.19.0]     |           -0.149988   |      0.45356      |         nan           |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| energySources[T.21.0]     |           -0.157302   |      0.463727     |         nan           |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| energySources[T.22.0]     |           -0.161774   |      0.41795      |         nan           |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| elevator[T.1]             |            0.00658242 |      0.781365     |          -0.000432945 |      0.98429      |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| sauna[T.1]                |            0.17705    |      0.141605     |           0.174691    |      0.140254     |            0.174629   |      0.139826     |            0.183422   |      0.0432718    |            0.183621   |      0.0428297    |            0.185591   |      0.0402081    |            0.184252   |      0.041521     |            0.187115   |      0.0383205    |             0.194371  |      0.0312516    |             0.195672   |       0.0303389    |             0.197303   |       0.029311     |             0.208812   |       0.0212304    |
| swimmingpool[T.1]         |           -0.0236529  |      0.900759     |           0.0211387   |      0.907757     |            0.021153   |      0.907595     |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| kitchenEqipped[T.1]       |            0.0573019  |      0.00432723   |           0.0543485   |      0.00477078   |            0.0543468  |      0.00472641   |            0.0545219  |      0.00442842   |            0.0547973  |      0.00413775   |            0.0547848  |      0.00411286   |            0.0546038  |      0.00421453   |            0.0539062  |      0.00468199   |             0.0543244 |      0.00439651   |             0.0528777  |       0.0055493    |             0.0524744  |       0.00601541   |             0.0575331  |       0.00245649   |
| cellar[T.1]               |            0.039799   |      0.0960984    |           0.0424134   |      0.0683482    |            0.0423915  |      0.0678779    |            0.0424459  |      0.0671793    |            0.0423763  |      0.0673333    |            0.0432043  |      0.0609851    |            0.0398842  |      0.0775166    |            0.0392967  |      0.081796     |             0.0392739 |      0.0821996    |             0.0468959  |       0.0333416    |             0.0449203  |       0.0416158    |             0.0525634  |       0.0157975    |
| storageRoom[T.1]          |            0.0349011  |      0.140599     |           0.0430513   |      0.0576381    |            0.0430222  |      0.0570054    |            0.0426849  |      0.0566195    |            0.0423052  |      0.0580507    |            0.0422196  |      0.0583238    |            0.0416534  |      0.0614827    |            0.0445428  |      0.0434044    |             0.0423163 |      0.0544037    |             0.0419288  |       0.0569294    |             0.0428876  |       0.0519109    |           nan          |     nan            |
| wheelchairAccessible[T.1] |           -0.0709139  |      0.292853     |          -0.0793845   |      0.211042     |           -0.0795211  |      0.207031     |           -0.0792341  |      0.207838     |           -0.0769711  |      0.216145     |           -0.0766174  |      0.217808     |           -0.0767527  |      0.216768     |           -0.0807242  |      0.192797     |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| assistedLiving[T.1]       |            0.0161905  |      0.84864      |           0.0219491   |      0.793215     |            0.0218077  |      0.793556     |            0.0215474  |      0.79568      |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| parkingSlots[T.1]         |            0.0234189  |      0.311853     |           0.0337338   |      0.125527     |            0.0336774  |      0.122533     |            0.0336409  |      0.1225       |            0.0331698  |      0.126098     |            0.0335269  |      0.12146      |            0.0334758  |      0.12184      |            0.0326546  |      0.130843     |             0.0317885 |      0.141449     |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| privateOutdoorSpace[T.1]  |           -0.00996568 |      0.652446     |          -0.0157368   |      0.462984     |           -0.015748   |      0.462022     |           -0.0157386  |      0.46181      |           -0.0154503  |      0.468999     |           -0.015639   |      0.463094     |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| chimney[T.1]              |           -0.0715189  |      0.349329     |          -0.0581145   |      0.388134     |           -0.0582021  |      0.385869     |           -0.0582461  |      0.384996     |           -0.0586507  |      0.381098     |           -0.0599237  |      0.369935     |           -0.0612664  |      0.358899     |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| guestToilet[T.1]          |            0.00148284 |      0.954961     |           0.0107107   |      0.667137     |            0.0106932  |      0.667111     |            0.010686   |      0.666988     |            0.0103363  |      0.676502     |          nan          |    nan            |          nan          |    nan            |          nan          |    nan            |           nan         |    nan            |           nan          |     nan            |           nan          |     nan            |           nan          |     nan            |
| attics[T.1]               |            0.0609406  |      0.136323     |           0.063703    |      0.109112     |            0.063683   |      0.108725     |            0.0633677  |      0.109269     |            0.0631654  |      0.109975     |            0.0637944  |      0.105924     |            0.0628919  |      0.110591     |            0.0624396  |      0.113081     |             0.0620637 |      0.115521     |             0.0681444  |       0.0827416    |             0.0655988  |       0.0952835    |             0.0652196  |       0.0981898    |
| log_livingSpace           |            0.976526   |      3.37581e-127 |           0.969513    |      1.52763e-134 |            0.969591   |      2.09599e-136 |            0.969469   |      8.70267e-137 |            0.969227   |      4.04194e-137 |            0.973305   |      7.49542e-149 |            0.969505   |      6.78496e-152 |            0.96485    |      8.64647e-155 |             0.961936  |      4.78058e-155 |             0.969912   |       1.39517e-160 |             0.965946   |       1.45709e-160 |             0.974112   |       2.51472e-164 |
| distanceToCenter          |           -0.00745443 |      0.143592     |          -0.00779693  |      0.10816      |           -0.00778335 |      0.104818     |           -0.00774639 |      0.105331     |           -0.00778366 |      0.103168     |           -0.007765   |      0.103672     |           -0.00836522 |      0.075094     |           -0.0085743  |      0.0677481    |            -0.0088475 |      0.0594093    |            -0.00797159 |       0.08711      |           nan          |     nan            |           nan          |     nan            |
| constructionYear          |            0.00208895 |      3.51009e-07  |           0.00203293  |      2.3694e-07   |            0.00203024 |      3.61953e-08  |            0.00203148 |      3.39235e-08  |            0.00203876 |      2.68364e-08  |            0.00204867 |      2.09698e-08  |            0.00200711 |      2.63901e-08  |            0.00204238 |      1.25673e-08  |             0.0020326 |      1.4886e-08   |             0.00213443 |       1.51745e-09  |             0.00203637 |       5.18922e-09  |             0.00214473 |       5.57396e-10  |
|    | Model    |   R-Squared |   Adj. R-Squared |
|---:|:---------|------------:|-----------------:|
|  0 | Model 1  |    0.854963 |         0.840263 |
|  0 | Model 2  |    0.852086 |         0.843441 |
|  0 | Model 3  |    0.852086 |         0.843779 |
|  0 | Model 4  |    0.852081 |         0.844111 |
|  0 | Model 5  |    0.85206  |         0.844424 |
|  0 | Model 6  |    0.852004 |         0.8447   |
|  0 | Model 7  |    0.851833 |         0.844853 |
|  0 | Model 8  |    0.851565 |         0.844905 |
|  0 | Model 9  |    0.851026 |         0.844673 |
|  0 | Model 10 |    0.850337 |         0.844287 |
|  0 | Model 11 |    0.849401 |         0.843646 |
|  0 | Model 12 |    0.848187 |         0.842719 |

In [12]:
# Nicht-Standardisiert
modelFormulas = [
    'price ~ livingSpace + distanceToCenter + location + constructionYear + energyClass + energySources + elevator + sauna + swimmingpool + kitchenEqipped + cellar + storageRoom + wheelchairAccessible + assistedLiving + parkingSlots + privateOutdoorSpace + chimney + guestToilet + attics',
    'price ~ livingSpace + distanceToCenter + location + constructionYear + energyClass + elevator + sauna + swimmingpool + kitchenEqipped + cellar + storageRoom + wheelchairAccessible + assistedLiving + parkingSlots + privateOutdoorSpace + chimney + guestToilet + attics',
    'price ~ livingSpace + distanceToCenter + location + constructionYear + energyClass + sauna + swimmingpool + kitchenEqipped + cellar + storageRoom + wheelchairAccessible + assistedLiving + parkingSlots + privateOutdoorSpace + chimney + guestToilet + attics',
    'price ~ livingSpace + distanceToCenter + location + constructionYear + energyClass + sauna + kitchenEqipped + cellar + storageRoom + wheelchairAccessible + assistedLiving + parkingSlots + privateOutdoorSpace + chimney + guestToilet + attics',
    'price ~ livingSpace + distanceToCenter + location + constructionYear + energyClass + sauna + kitchenEqipped + cellar + storageRoom + wheelchairAccessible + parkingSlots + privateOutdoorSpace + chimney + guestToilet + attics',
    'price ~ livingSpace + distanceToCenter + location + constructionYear + energyClass + sauna + kitchenEqipped + cellar + storageRoom + wheelchairAccessible + parkingSlots + privateOutdoorSpace + chimney + attics',
    'price ~ livingSpace + distanceToCenter + location + constructionYear + energyClass + sauna + kitchenEqipped + cellar + storageRoom + wheelchairAccessible + parkingSlots + chimney + attics',
    'price ~ livingSpace + distanceToCenter + location + constructionYear + energyClass + sauna + kitchenEqipped + cellar + storageRoom + wheelchairAccessible + parkingSlots + attics',
    'price ~ livingSpace + distanceToCenter + location + constructionYear + energyClass + sauna + kitchenEqipped + cellar + storageRoom + parkingSlots + attics',
    'price ~ livingSpace + distanceToCenter + location + constructionYear + energyClass + sauna + kitchenEqipped + cellar + storageRoom + attics',
    'price ~ livingSpace + location + constructionYear + energyClass + sauna + kitchenEqipped + cellar + storageRoom + attics',
    'price ~ livingSpace + location + constructionYear + energyClass + sauna + kitchenEqipped + cellar + attics',
]

In [13]:
# DataFrame für Koeffizienten und p-Werte
modelSummary = pd.DataFrame()

# DataFrame für R-Quadrat und angepasstes R-Quadrat
modelFitSummary = pd.DataFrame(columns=['Model', 'R-Squared', 'Adj. R-Squared'])

# Index setzen
firstModel = smf.ols(modelFormulas[0], data=df_train_price).fit()
modelSummary['Coefficient_Model_1'] = firstModel.params
modelSummary['P-Value_Model_1'] = firstModel.pvalues
firstFitSummary = pd.DataFrame({'Model': ['Model 1'],
                                  'R-Squared': [firstModel.rsquared],
                                  'Adj. R-Squared': [firstModel.rsquared_adj]})
modelFitSummary = pd.concat([modelFitSummary, firstFitSummary])

# Restliche  Modelle durchlaufen
for i, formula in enumerate(modelFormulas[1:], start=2):
    model = smf.ols(formula, data=df_train_price).fit()
    modelSummary[f'Coefficient_Model_{i}'] = model.params
    modelSummary[f'P-Value_Model_{i}'] = model.pvalues
    fitSummary = pd.DataFrame({'Model': [f'Model {i}'],
                                'R-Squared': [model.rsquared],
                                'Adj. R-Squared': [model.rsquared_adj]})
    modelFitSummary = pd.concat([modelFitSummary, fitSummary])

# Ergebnisse anzeigen
print(modelSummary.T)
print(modelFitSummary.T)


modelSummary_markdown = modelSummary.to_markdown()
modelFitSummary_markdown = modelFitSummary.to_markdown()

C:\Users\papad\AppData\Local\Temp\ipykernel_12804\3725840371.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  modelFitSummary = pd.concat([modelFitSummary, firstFitSummary])


                         Intercept  location[T.2]  location[T.3]  \
Coefficient_Model_1  -1.293473e+06  -32680.910764  -5.044291e+04   
P-Value_Model_1       7.134716e-06       0.004428   6.863959e-07   
Coefficient_Model_2  -1.158509e+06  -34237.322264  -5.110693e+04   
P-Value_Model_2       2.912527e-05       0.002076   1.646320e-07   
Coefficient_Model_3  -1.184358e+06  -34051.010741  -5.095542e+04   
P-Value_Model_3       5.822296e-06       0.002133   1.662965e-07   
Coefficient_Model_4  -1.163880e+06  -34708.976789  -5.106885e+04   
P-Value_Model_4       9.496734e-06       0.001874   1.884102e-07   
Coefficient_Model_5  -1.158517e+06  -34599.747128  -5.080282e+04   
P-Value_Model_5       9.675902e-06       0.001904   1.854529e-07   
Coefficient_Model_6  -1.184393e+06  -33578.895542  -5.161559e+04   
P-Value_Model_6       6.352129e-06       0.002622   1.259438e-07   
Coefficient_Model_7  -1.147289e+06  -33777.915860  -5.141389e+04   
P-Value_Model_7       9.843618e-06       0.00246

In [14]:
print(modelSummary_markdown)
print(modelFitSummary_markdown)

|                           |   Coefficient_Model_1 |   P-Value_Model_1 |   Coefficient_Model_2 |   P-Value_Model_2 |   Coefficient_Model_3 |   P-Value_Model_3 |   Coefficient_Model_4 |   P-Value_Model_4 |   Coefficient_Model_5 |   P-Value_Model_5 |   Coefficient_Model_6 |   P-Value_Model_6 |   Coefficient_Model_7 |   P-Value_Model_7 |   Coefficient_Model_8 |   P-Value_Model_8 |   Coefficient_Model_9 |   P-Value_Model_9 |   Coefficient_Model_10 |   P-Value_Model_10 |   Coefficient_Model_11 |   P-Value_Model_11 |   Coefficient_Model_12 |   P-Value_Model_12 |
|:--------------------------|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|-----------

|                           |   Coefficient_Model_1 |   P-Value_Model_1 |   Coefficient_Model_2 |   P-Value_Model_2 |   Coefficient_Model_3 |   P-Value_Model_3 |   Coefficient_Model_4 |   P-Value_Model_4 |   Coefficient_Model_5 |   P-Value_Model_5 |   Coefficient_Model_6 |   P-Value_Model_6 |   Coefficient_Model_7 |   P-Value_Model_7 |   Coefficient_Model_8 |   P-Value_Model_8 |   Coefficient_Model_9 |   P-Value_Model_9 |   Coefficient_Model_10 |   P-Value_Model_10 |   Coefficient_Model_11 |   P-Value_Model_11 |   Coefficient_Model_12 |   P-Value_Model_12 |
|:--------------------------|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|----------------------:|------------------:|-----------------------:|-------------------:|-----------------------:|-------------------:|-----------------------:|-------------------:|
| Intercept                 |          -1.29347e+06 |      7.13472e-06  |          -1.15851e+06 |      2.91253e-05  |          -1.18436e+06 |      5.8223e-06   |          -1.16388e+06 |      9.49673e-06  |          -1.15852e+06 |      9.6759e-06   |          -1.18439e+06 |      6.35213e-06  |          -1.14729e+06 |      9.84362e-06  |          -1.14808e+06 |      8.46294e-06  |          -1.15301e+06 |       7.67903e-06 |            -1.2271e+06 |       1.31762e-06  |           -1.14453e+06 |       5.57647e-06  |           -1.28363e+06 |       3.68827e-07  |
| location[T.2]             |      -32680.9         |      0.00442759   |      -34237.3         |      0.00207593   |      -34051           |      0.00213282   |      -34709           |      0.00187387   |      -34599.7         |      0.00190418   |      -33578.9         |      0.00262166   |      -33777.9         |      0.00246895   |      -33771.5         |      0.00244244   |      -33824.2         |       0.00239894  |        -33382.1        |       0.00275754   |       -32511.9         |       0.00368327   |       -29887.5         |       0.00814154   |
| location[T.3]             |      -50442.9         |      6.86396e-07  |      -51106.9         |      1.64632e-07  |      -50955.4         |      1.66296e-07  |      -51068.9         |      1.8841e-07   |      -50802.8         |      1.85453e-07  |      -51615.6         |      1.25944e-07  |      -51413.9         |      1.3949e-07   |      -51396.9         |      1.2959e-07   |      -51043           |       1.52259e-07 |        -51858.6        |       9.68056e-08  |       -56911           |       2.60553e-09  |       -57810.1         |       2.2463e-09   |
| location[T.4]             |      -46501.4         |      3.98299e-07  |      -44024.4         |      7.69534e-07  |      -44037.7         |      7.44403e-07  |      -44463.8         |      6.91015e-07  |      -44233.5         |      6.89259e-07  |      -44445.8         |      6.55546e-07  |      -43332.9         |      1.00268e-06  |      -43312.3         |      9.10789e-07  |      -43928.2         |       5.89512e-07 |        -44727.3        |       3.66251e-07  |       -47757.6         |       4.92484e-08  |       -45059.7         |       3.13007e-07  |
| energyClass[T.2]          |      -55010.1         |      0.0744255    |      -76186.6         |      0.00110464   |      -76515.7         |      0.00102433   |      -70950.9         |      0.00237577   |      -70646.8         |      0.00242886   |      -73508.2         |      0.00162674   |      -73696.6         |      0.00158195   |      -73694.8         |      0.00156405   |      -70358.7         |       0.00220304  |        -70810.2        |       0.00209089   |       -62312.6         |       0.00629345   |       -73371.9         |       0.00134952   |
| energyClass[T.3]          |      -64918.7         |      0.0195534    |      -78520.7         |      1.11878e-05  |      -78716.1         |      1.02759e-05  |      -78877.1         |      1.1293e-05   |      -78997.7         |      1.06767e-05  |      -80804.4         |      6.95267e-06  |      -79896.3         |      8.52716e-06  |      -79949.5         |      7.37215e-06  |      -79752.9         |       7.70304e-06 |        -81637.1        |       4.60656e-06  |       -71487.6         |       3.66167e-05  |       -72705.8         |       3.33682e-05  |
| energyClass[T.4]          |     -122863           |      9.51155e-06  |     -141447           |      1.08595e-14  |     -141813           |      7.54891e-15  |     -142177           |      9.58083e-15  |     -142423           |      7.62264e-15  |     -143806           |      4.88614e-15  |     -144336           |      3.81928e-15  |     -144316           |      3.44381e-15  |     -144601           |       2.98358e-15 |       -143614          |       4.62568e-15  |      -140919           |       1.59387e-14  |      -148259           |       1.07756e-15  |
| energyClass[T.5]          |     -142835           |      2.14971e-07  |     -160370           |      2.82019e-19  |     -160657           |      1.98485e-19  |     -159983           |      4.45835e-19  |     -159984           |      4.11029e-19  |     -163044           |      8.74916e-20  |     -162840           |      9.50524e-20  |     -162828           |      8.55134e-20  |     -162908           |       8.02653e-20 |       -162432          |       1.09233e-19  |      -157600           |       8.6034e-19   |      -163115           |       1.15472e-19  |
| energyClass[T.6]          |     -138257           |      9.19578e-07  |     -156323           |      2.04764e-16  |     -156565           |      1.61324e-16  |     -155840           |      3.30334e-16  |     -155768           |      3.16061e-16  |     -158561           |      9.91639e-17  |     -159080           |      7.76278e-17  |     -159055           |      6.74927e-17  |     -158652           |       7.65921e-17 |       -159257          |       6.38623e-17  |      -155306           |       3.32003e-16  |      -161908           |       3.39676e-17  |
| energyClass[T.7]          |     -132698           |      1.05661e-05  |     -151898           |      5.92068e-13  |     -152745           |      2.42773e-13  |     -152690           |      3.49632e-13  |     -152648           |      3.35519e-13  |     -155652           |      1.21286e-13  |     -156919           |      7.07483e-14  |     -156961           |      5.72929e-14  |     -157133           |       5.30334e-14 |       -157115          |       5.72231e-14  |      -149145           |       5.27052e-13  |      -153496           |       2.01319e-13  |
| energyClass[T.8]          |     -119400           |      0.000396214  |     -133075           |      2.79154e-07  |     -133873           |      1.95518e-07  |     -133040           |      2.77048e-07  |     -132941           |      2.74306e-07  |     -136094           |      1.48267e-07  |     -136813           |      1.26477e-07  |     -136791           |      1.21594e-07  |     -136750           |       1.21725e-07 |       -137429          |       1.09424e-07  |      -133154           |       2.76405e-07  |      -143019           |       4.31834e-08  |
| energyClass[T.9]          |     -101548           |      0.0795406    |     -117785           |      0.0271923    |     -118262           |      0.0263516    |     -117505           |      0.0283818    |     -117481           |      0.0282573    |     -120929           |      0.0242881    |     -122430           |      0.0225331    |     -122401           |      0.0223976    |     -121872           |       0.0229427   |       -121263          |       0.0238145    |      -114081           |       0.0339809    |      -122269           |       0.0246324    |
| energySources[T.2.0]      |       -9910.13        |      0.290248     |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| energySources[T.3.0]      |      -26280.6         |      0.0229317    |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| energySources[T.4.0]      |       -2060.27        |      0.914206     |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| energySources[T.5.0]      |       53650.1         |      0.150539     |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| energySources[T.6.0]      |       -4660.58        |      0.890167     |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| energySources[T.7.0]      |       27358.7         |      0.306114     |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| energySources[T.8.0]      |       -2960.22        |      0.922304     |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| energySources[T.9.0]      |      -53633.5         |      0.134203     |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| energySources[T.10.0]     |      -19656.9         |      0.538697     |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| energySources[T.11.0]     |       41406.5         |      0.58304      |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| energySources[T.12.0]     |      -40935.4         |      0.422934     |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| energySources[T.13.0]     |       30869.6         |      0.58562      |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| energySources[T.14.0]     |       30791.8         |      0.666792     |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| energySources[T.15.0]     |      -24914.7         |      0.72452      |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| energySources[T.16.0]     |           2.40067e-11 |      0.429006     |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| energySources[T.18.0]     |      -64244.8         |      0.397014     |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| energySources[T.19.0]     |        -614.105       |      0.993073     |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| energySources[T.21.0]     |      -46117.6         |      0.543409     |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| energySources[T.22.0]     |      -33670.1         |      0.633653     |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| elevator[T.1]             |        8244.96        |      0.327637     |        2206.67        |      0.778732     |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| sauna[T.1]                |      233278           |      8.37915e-08  |      229998           |      9.78358e-08  |      230422           |      8.8521e-08   |      156286           |      2.29198e-06  |      156195           |      2.26687e-06  |      159305           |      1.49249e-06  |      158439           |      1.68028e-06  |      158480           |      1.59228e-06  |      156623           |       1.984e-06   |        156841          |       1.9808e-06   |       157146           |       2.11189e-06  |       165514           |       7.65138e-07  |
| swimmingpool[T.1]         |     -172487           |      0.010644     |     -176714           |      0.00697989   |     -176917           |      0.00685644   |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| kitchenEqipped[T.1]       |       10610.1         |      0.132588     |       10872.8         |      0.11152      |       10892           |      0.11052      |        9440.36        |      0.168048     |        9330.12        |      0.171935     |        9087.92        |      0.184502     |        9079.92        |      0.184885     |        9073.07        |      0.184485     |        8951.3         |       0.190247    |          8338.13       |       0.222167     |         8258.87        |       0.228798     |        11321           |       0.100498     |
| cellar[T.1]               |        4551.43        |      0.590275     |        7111.92        |      0.391046     |        7219.06        |      0.382977     |        6899.79        |      0.407507     |        6928.16        |      0.405083     |        8215.35        |      0.32341      |        6623.91        |      0.417148     |        6615.43        |      0.416947     |        6638.09        |       0.415248    |          9518.07       |       0.230328     |         8535.32        |       0.28363      |        13632.6         |       0.0855158    |
| storageRoom[T.1]          |       20172.7         |      0.0151028    |       23747.6         |      0.00307739   |       23871           |      0.00285844   |       26665.8         |      0.000858671  |       26820.2         |      0.00076985   |       27211.7         |      0.000665662  |       26737           |      0.000806124  |       26767           |      0.000708111  |       27365.2         |       0.000511732 |         27431          |       0.000504732  |        27646.6         |       0.000484805  |          nan           |     nan            |
| wheelchairAccessible[T.1] |       18735           |      0.432443     |       22140.1         |      0.328128     |       22843.9         |      0.309538     |       20253           |      0.370397     |       19364.3         |      0.386636     |       20124.9         |      0.369579     |       19926.9         |      0.374282     |       19883.2         |      0.373739     |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| assistedLiving[T.1]       |      -14369.8         |      0.632232     |      -11476.2         |      0.70081      |      -10762.4         |      0.717348     |       -8449.62        |      0.777635     |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| parkingSlots[T.1]         |        6877.51        |      0.400606     |       10310.2         |      0.18946      |       10602.4         |      0.173        |       10744.4         |      0.170219     |       10928           |      0.161183     |       11689.5         |      0.134618     |       11538.3         |      0.139643     |       11530.3         |      0.139245     |       11740.2         |       0.131939    |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| privateOutdoorSpace[T.1]  |       -4532.9         |      0.556663     |       -7631.12        |      0.311847     |       -7597.54        |      0.313436     |       -7716.74        |      0.309201     |       -7824.06        |      0.301442     |       -7574.09        |      0.318456     |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| chimney[T.1]              |        2929.61        |      0.914154     |        1916.3         |      0.936801     |        2405.48        |      0.920431     |        2057.21        |      0.932382     |        2205.56        |      0.92743      |         245.472       |      0.991928     |        -703.235       |      0.976861     |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| guestToilet[T.1]          |       10634.9         |      0.241983     |       16723.9         |      0.054783     |       16796.4         |      0.0534021    |       16607.4         |      0.0578092    |       16748.5         |      0.0550826    |         nan           |    nan            |         nan           |    nan            |         nan           |    nan            |         nan           |     nan           |           nan          |     nan            |          nan           |     nan            |          nan           |     nan            |
| attics[T.1]               |       25635.3         |      0.0767224    |       25268.5         |      0.0748827    |       25365           |      0.0734113    |       28018.7         |      0.049054     |       28098.8         |      0.0481457    |       29213.8         |      0.0403677    |       28731.8         |      0.0436465    |       28725.9         |      0.0434434    |       28829.5         |       0.0426423   |         31121          |       0.0280089    |        29740.3         |       0.036438     |        29766.5         |       0.0385227    |
| livingSpace               |        3904.28        |      7.29783e-141 |        3891           |      4.52607e-146 |        3886.63        |      4.90605e-148 |        3908.86        |      1.56469e-148 |        3909.91        |      5.54784e-149 |        3968.91        |      8.44871e-159 |        3956.41        |      7.60306e-160 |        3955.77        |      1.10888e-164 |        3963.58        |       5.875e-166  |          3995.13       |       1.28993e-171 |         3986.67        |       8.55798e-171 |         4020.84        |       2.09538e-171 |
| distanceToCenter          |       -3282.67        |      0.0694943    |       -3779.14        |      0.0295024    |       -3852.01        |      0.0247127    |       -4143.05        |      0.0162646    |       -4127.51        |      0.0165045    |       -4041.33        |      0.0191866    |       -4353.03        |      0.0103582    |       -4356.07        |      0.0100804    |       -4280.2         |       0.0113303   |         -3921.58       |       0.0191995    |          nan           |     nan            |          nan           |     nan            |
| constructionYear          |         736.726       |      3.9507e-07   |         675.096       |      1.48089e-06  |         689.005       |      1.4946e-07   |         678.195       |      2.72549e-07  |         675.286       |      2.73483e-07  |         688.231       |      1.70462e-07  |         668.384       |      2.72368e-07  |         668.804       |      2.1886e-07   |         671.012       |       1.97763e-07 |           707.793      |       2.53335e-08  |          658.759       |       1.57107e-07  |          731.407       |       6.02344e-09  |
|    | Model    |   R-Squared |   Adj. R-Squared |
|---:|:---------|------------:|-----------------:|
|  0 | Model 1  |    0.870176 |         0.857018 |
|  0 | Model 2  |    0.865486 |         0.857625 |
|  0 | Model 3  |    0.865463 |         0.857908 |
|  0 | Model 4  |    0.863319 |         0.855955 |
|  0 | Model 5  |    0.863296 |         0.85624  |
|  0 | Model 6  |    0.862209 |         0.855408 |
|  0 | Model 7  |    0.861914 |         0.855409 |
|  0 | Model 8  |    0.861914 |         0.855717 |
|  0 | Model 9  |    0.86168  |         0.855781 |
|  0 | Model 10 |    0.861008 |         0.855389 |
|  0 | Model 11 |    0.859375 |         0.854001 |
|  0 | Model 12 |    0.855689 |         0.850492 |

In [15]:
# Formula 
formula = 'np.log(price) ~ log_livingSpace + location + constructionYear + energyClass + sauna + kitchenEqipped + cellar'

# Fit 
model = smf.ols(formula=formula, data=df_train_price).fit()

# Summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(price)   R-squared:                       0.847
Model:                            OLS   Adj. R-squared:                  0.842
Method:                 Least Squares   F-statistic:                     164.0
Date:                Tue, 02 Jan 2024   Prob (F-statistic):          2.75e-181
Time:                        23:34:15   Log-Likelihood:                 112.34
No. Observations:                 490   AIC:                            -190.7
Df Residuals:                     473   BIC:                            -119.4
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               4.4496    

In [16]:
# Formula 
formula = 'price ~ livingSpace + location + constructionYear + energyClass + sauna + kitchenEqipped + cellar'

# Fit 
model = smf.ols(formula=formula, data=df_train_price).fit()

# Summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.854
Model:                            OLS   Adj. R-squared:                  0.849
Method:                 Least Squares   F-statistic:                     173.4
Date:                Tue, 02 Jan 2024   Prob (F-statistic):          3.94e-186
Time:                        23:34:41   Log-Likelihood:                -6163.8
No. Observations:                 490   AIC:                         1.236e+04
Df Residuals:                     473   BIC:                         1.243e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept            -1.31e+06   2

Beide Modelle können als statistisch robust betrachtet werden. Obwohl das nicht log-Modell im Bezug auf R² und adjusted R² leicht besser ist, weist das log-Modell bei AIC, BIC und der Durbin-Watson-Statistik Vorteile auf. Deshalb argumentieren wir, dass das log-Modell insgesamt vorzuziehen ist.